# Project for Econ 622, Jan Rosa: package HA_solver

In this project, I developed the package to solve two basic (workhorse) macroeconomic models:

- Neoclassical stochastic growth model (representative agent model): I solve it by 2D fixed point problem
- Aiyagari model: model with hereogenous agents which face the idiosycratic earnings risks
- I planned to add also the model for the soverign default (which I will add anyway by the end of the year, but after deadline), I was not able to finish that model 
- I have old code for the Hugget model, but I do not test it for this project

## General way of solving the models  
I divided the macroeconomic models on the two cathegories:

- Heterogenous agent models
- Representative agent models

For the second cathegory, the general way of solving is very simple: defined by the genral function SolveRAEq. We firstly solve the model: find polcy function for the model. As there are a lot of possible way to solve the model, (value function, projection method- which we use it now), we defined it as a general function SolveRAP. Next, we simulate the model, by general function SolveDistr, last we use the function to save equilibrium. 

For the Heterogenous agent models, the general procedure is more complicated, defined in the genral (using generic programming) function SolveHAEq. The aim of the algorithm, is to find a fixed/ stationary point (whatever defined) of the  equilirbirum for some variable which defines it (for example an interest rate), which we will then call equilibrium variable. The main steps of the algorithm are:

- Solve individual agent problem: defined by input SolveAgP
- Aggregate the equilibrium/ find the distribution of agent: defined by input SolveDistr
- Update the equilibrium variable: UpdateGEq
- Check the convergence of the equilibrium variable: ConvSolParam

## Neoclassical growth model solution
In the module NC_growth, we solved the Neoclassical growth stochastic growth model. We also check that :
$$
\begin{aligned}
\max_{C_t,K_t}& \sum_{t=0}^{\infty}\beta^tU(C_t)\\
wrt\:& C_t +K_{t+1} = A_tK_t^{\alpha}+(1-\delta) K_t\\
&\ln(A_t) = \rho\ln(A_{t-1})+\epsilon_t\\
&\epsilon_t \sim N(0, \sigma_{A})\\
&U(C)=\frac{C^{1-\sigma}}{1-\sigma}
\end{aligned}
$$
The FOC for this problem is:
$$
1 = \beta E(1+\alpha A'K'^{\alpha-1}-\delta){C'}^{-\sigma}/C^{-\sigma}
$$
From this, we defive the fixed problem:
$$
K' = \beta E(1+\alpha A'K'^{\alpha-1}-\delta){C'}^{-\sigma}/C^{-\sigma} K'
$$
We solve this problem using the general method, defined in the previous part. We solved the model by the Smolyak algorithm. That is we defined 2D grid:$(x_j, y_j)$ where $x_j$ is a log of capital value, $y_j$ is the log of TFP. The gird was transformed the the Smolyak grid (special set of points for fast multidimensional interpolation), and the tensor product of the interpolating Chebyschev polynomials (interpolated on the grid). We compute future possible $y_j'$, and discrete probability of drawing that value by Gauss quadrature: $w_i$. Next, we solve the model in the steps:
- Guess the future capital policy values on the grid: $K'(x_j, y_j)$, and compute the all possible next period log of TFP values $y_{j}'$, and next period capital interpolations $K'(log(K'(x_j, y_j)), y_j')$. 
- Using a budget constraint, find values of consumption: C, C'
- Find next guess for of $K'(x_j, y_j)$ by solving the fixed equation (obtained from FOC):
$$
\begin{aligned}
K_{new}' &= \beta E(1+\alpha A'K'^{\alpha-1}-\delta){C'}^{-\sigma}/C^{-\sigma}K'(x_j, y_j)\\
&\approx  \beta \sum_{i=1}^M w_{i}(1+\alpha A'K'^{\alpha-1}-\delta){C'((log(K'(x_j, y_j)), y_j'))}^{-\sigma}/C^{-\sigma}(x_j,y_j)K'(x_j, y_j)
\end{aligned} 
$$      
For more information on Smolyak algorithm: Judd, Maliar, Maliar, Valero (2014). The described procedure is impemented in the function Solve_Smolyak. From this function we derive the evaluation function, next we simulate the model using the Monte Carlo, using function simulate_smolyak_NGS.

## Ayiagari model solution
Here, I do not describe the Ayiagari model formally (with equilibrium definition, all market clearing conditions), just write the main points. In the Ayiagari model, the consumers face idiosyncratic earinings risk, that is they solve the problem:

$$
\begin{aligned}
\max_{C_t,A_t}& \sum_{t=0}^{\infty}\beta^tU(C_t)\\
wrt\:& C_t +A_{t+1} = (1+r)A_t+W_t\\
&\ln(w_t) = \rho\ln(w_{t-1})+\epsilon_t\\
&\epsilon_t \sim N(0, \sigma_{w})\\
&U(C)=\frac{C^{1-\sigma}}{1-\sigma}
\end{aligned}
$$
Where r is a interest rate, wage is $w_{t}$, the consumer takes them as given. We solve this problem by endogenous grid method, in the code, it is a method SolveAgP_Ayiagari_EGM. The stationary measure is computed in the function SolveDistr_Ayiagari_Iter.  

There is no aggregate risk, in this economy, the production is defined as in the purely neoclassical growth model with the perfectly elastic labor supply. That is $r+\delta= \alpha K^{\alpha-1}-1.0$, $w=1.0$. The capital is derived from the market clearing condition:
$$
\begin{aligned}
\int A'd\lambda(A,w) = K 
\end{aligned}
$$

Where $\lambda$ is an invariant measure.

The equilibrium is defined (given the parameters) by the interest rate r. Thus, the equilibrium variable that we look for is r. We solve the whole model using the function Find_eq_Ayiagari, for the defined type of the model. 

## Tests
We provide the set of tests for most of the functions for the Ayiagari model and Neoclassical stochastoc growth model. For Ayiagari model we also compre the solution values with the ones orginally obtained by Ayiagari 1994.